In [3]:
# importing libraries/
import os
import pandas as pd
import requests
import schedule
import time
import logging
from datetime import datetime
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
from kaggle.api.kaggle_api_extended import KaggleApi



In [1]:
!aws s3 ls


2025-03-08 21:03:57 dmml-bank-churn-data


File association not found for extension .py


In [2]:
# Initialize DVC in the project (this creates a .dvc folder for tracking data files)
# !dvc init


ERROR: failed to initiate DVC - '.dvc' exists. Use `-f` to force.


#### Setting Kaggle API 

In [4]:
# Set Kaggle API credentials
os.environ["KAGGLE_CONFIG_DIR"] = "C:\\Users\\LENOVO\\.kaggle"

# Verify Kaggle setup
!kaggle datasets list


ref                                                               title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
b'atharvasoundankar/chocolate-sales                                 Chocolate Sales Data \xf0\x9f\x93\x8a\xf0\x9f\x8d\xab                            63KB  2025-03-07 05:44:03           2521         35  1.0              '
abdulmalik1518/mobiles-dataset-2025                               Mobiles Dataset (2025)                             20KB  2025-02-18 06:50:24           9332        164  1.0              
ignacioazua/life-expectancy                                       Life Expectancy                                     3KB  2025-03-04 06:16:35            961         22  1.0              
mahmoudelhemaly/students-gr

#### Fetching data from Two sources
1. Kaggle Dataset
2. Rest API
3. Generating log report


In [5]:
#Only fetching the data
def setup_logging():
    """Sets up logging for data ingestion."""
    log_file = "logs/data_ingestion.log"
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logging.basicConfig(
        filename=log_file,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
    )

def fetch_kaggle_data():
    """Downloads data from Kaggle and saves it locally."""
    kaggle_dataset = "rangalamahesh/bank-churn"
    download_path = "raw_data/"
    os.makedirs(download_path, exist_ok=True)
    
    try:
        api = KaggleApi()
        api.authenticate()
        api.dataset_download_files(kaggle_dataset, path=download_path, unzip=True)
        
        # Locate and rename CSV file
        for file in os.listdir(download_path):
            if file.endswith(".csv"):
                os.rename(os.path.join(download_path, file), os.path.join(download_path, "kaggle_data.csv"))
                logging.info("Successfully downloaded Kaggle dataset.")
                return
        
        logging.error("Kaggle dataset downloaded but no CSV file found.")
    except Exception as e:
        logging.error(f"Failed to download Kaggle dataset: {e}")

def fetch_api_data():
    """Fetches data from API and saves it as CSV locally."""
    api_url = "https://bank-customer-churn-data.onrender.com/data"
    file_path = "raw_data/api_data.csv"
    os.makedirs("raw_data", exist_ok=True)
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        
        data = response.json()
        df = pd.DataFrame(data)
        df.to_csv(file_path, index=False)
        
        logging.info("Successfully fetched and saved API data.")
    except requests.RequestException as e:
        logging.error(f"Failed to fetch API data: {e}")

def main():
    """Main function to execute the ingestion process."""
    logging.info("Starting data ingestion...")
    fetch_kaggle_data()
    fetch_api_data()
    logging.info("Data ingestion completed successfully.")

if __name__ == "__main__":
    setup_logging()
    main()


Dataset URL: https://www.kaggle.com/datasets/rangalamahesh/bank-churn
